# Calculating proportion of human encroachment in a grid
This notebook uses the subsetted gHM by the 4 anthromes to calculate the percentage of cell from the master grid that is encroached for each type of activity: Urban, Rainfed Agriculture, Rainfed Irrigated and Rangeland. Data has been classified in a previous notebook. 

NOTE: This is only for the terrestrial environment

In [1]:
import arcpy
from arcpy.sa import *
import arcgis
from arcgis.gis import GIS
import datetime

In [2]:
tmp = []
with open(".env") as f:
    for line in f:
        tmp.append(line)
aol_password = tmp[0].split("=")[1].split("\n")[0]
aol_username = tmp[1].split("=")[1].split("\n")[0]

In [7]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password)

In [8]:
grid_id = "a52aa51088f84eb8b95f90be8fa77175"
grid_item=gis.content.get(grid_id)

In [10]:
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\Human_pressure_outputs.gdb"
ghm = "..\\gHM_extended_maskGADM36_cea_20191127.tif"
grid = grid_item.layers[0].url
ZSaTGrid = "ZSaTGrid"

Zonal Statistics calculates the `COUNT` by default. 

In [11]:
start = datetime.datetime.now()
outZSaT = ZonalStatisticsAsTable(grid,
                                 "CELL_ID", 
                                 ghm,
                                 ZSaTGrid, "DATA", "ALL")
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:02:13.088892


In [13]:
ghm_class_rasters = arcpy.ListRasters("gHM*")
for raster in ghm_class_rasters:
    print(raster)

gHM_Urban
gHM_Rangeland
gHM_Rainfed
gHM_Irrigated


In [14]:
for raster in ghm_class_rasters:
    anthrome_class=raster.split(f"_")[1]
    outTableName = f"ZSat_{anthrome_class}"
    start = datetime.datetime.now()
    outZSat = ZonalStatisticsAsTable(grid,
                                     "CELL_ID", 
                                     raster,
                                     outTableName, "DATA", "ALL")
    end = datetime.datetime.now() - start
    print(f"{anthrome_class} finished in {end}")

Urban finished in 0:01:46.813637
Rangeland finished in 0:01:26.722224
Rainfed finished in 0:01:23.530187
Irrigated finished in 0:01:23.113007


- Merge the tables 5 tables should do mapping fields, keeping only the appropriate fields and renaming accordingly. 
- Then the `None` values should be replace by `0`.
- Finally calculate the proportions

In [15]:
zsa_tables = arcpy.ListTables("ZSat_*")
ZSaTGrid = "ZSaTGrid"
for tbl in zsa_tables:
    pos = len(arcpy.ListFields(ZSaTGrid))
    start = datetime.datetime.now()
    arcpy.management.JoinField(ZSaTGrid, "CELL_ID", tbl, "CELL_ID", "COUNT")
    anthrome_class=tbl.split("_")[1]
    newField_name = f"COUNT_{anthrome_class}" 
    oldField_name = arcpy.ListFields(ZSaTGrid)[pos].name 
    arcpy.AlterField_management(ZSaTGrid, oldField_name, newField_name, newField_name)
    end = datetime.datetime.now() - start
    print(f"{tbl} finished in {end}")

ZSat_Urban finished in 0:00:06.085116
ZSat_Rangeland finished in 0:00:06.743332
ZSat_Rainfed finished in 0:00:06.505298
ZSat_Irrigated finished in 0:00:05.566216


We have to replace the `None` with `0`s.

In [16]:
fieldObs = arcpy.ListFields(ZSaTGrid)  
fieldNames = []
for field in fieldObs:
    fieldNames.append(field.name)
del fieldObs
fieldCount = len(fieldNames)

In [18]:
start = datetime.datetime.now()
with arcpy.da.UpdateCursor(ZSaTGrid, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = "0"  
        curU.updateRow(rowU)
del curU
end = datetime.datetime.now() - start
print(f"Finished in {end}")

Finished in 0:00:02.226058


We have to create new fields (one for each type of encroachment) and calculate the proportion. 

In [19]:
for tbl in zsa_tables:
    anthrome_class = tbl.split("_")[1]
    prop_name = f"prop_{anthrome_class}"
    start = datetime.datetime.now()
    arcpy.management.AddField(ZSaTGrid, 
                          prop_name, 
                          "DOUBLE", 
                          None, None, None, 
                          prop_name, "NULLABLE", "NON_REQUIRED", '')
    COUNT_name = f"COUNT_{anthrome_class}" 
    py_formula = f"(!{COUNT_name}! / !COUNT!) * 100"
    arcpy.management.CalculateField(ZSaTGrid, 
                                prop_name, 
                                py_formula, 
                                "PYTHON3", '') 
    end = datetime.datetime.now() - start
    print(f"{anthrome_class} finished in {end}")

Urban finished in 0:00:11.375532
Rangeland finished in 0:00:11.150509
Rainfed finished in 0:00:11.844532
Irrigated finished in 0:00:11.358512


In [21]:
fieldObs = arcpy.ListFields(ZSaTGrid) 
for field in fieldObs:
    print(field.name)

OBJECTID
CELL_ID
ZONE_CODE
COUNT
AREA
MIN
MAX
RANGE
MEAN
STD
SUM
COUNT_Urban
COUNT_Rangeland
COUNT_Rainfed
COUNT_Irrigated
prop_Urban
prop_Rangeland
prop_Rainfed
prop_Irrigated


In [22]:
arcpy.TableToTable_conversion(ZSaTGrid, "..", "human_prop.csv")

<Result '..\\human_prop.csv'>

The table has been uploaded to arcgis online via the browser. 